Kapitel 8 aus [Data Science from Scratch](https://www.amazon.com/Data-Science-Scratch-Principles-Python/dp/149190142X).

Wir minimieren die Summe der Quadrate als Beispielfunktion. Lösung Nullvektor 

In [125]:
def sum_of_squares(v):
    """computes the sum of squared elements in v"""
    return sum(v_i ** 2 for v_i in v)

Wenn keine analytische Ableitung vorhanden, numerisch schätzen. Echten Limes kann Python nicht.

In [126]:
def difference_quotient(f, x, h=0.00001):
    return (f(x + h) - f(x)) / h

In [127]:
def partial_difference_quotient(f, v, i, h=0.00001):
    """compute the ith partial difference quotient of f at v"""
    w = [v_j + (h if j == i else 0)    # add h to just the ith element of v
         for j, v_j in enumerate(v)]

    return (f(w) - f(v)) / h

In [128]:
def estimate_gradient(f, v, h=0.00001):
    return [partial_difference_quotient(f, v, i, h)
            for i, _ in enumerate(v)]

Achtung sehr langsam, da f in jeder Iteration 2n-fach berechnet wird. Wenn möglich Ableitung daher besser analytisch bereitstellen.

In [129]:
def sum_of_squares_gradient(v):
    return [2 * v_i for v_i in v]

Falls Zielfunktion nicht für Werte von X definiert, Unendlich zurückgeben.

In [78]:
def safe(f):
    """return a new function that's the same as f,
    except that it outputs infinity whenever f produces an error"""
    def safe_f(*args, **kwargs):
        try:
            return f(*args, **kwargs)
        except:
            return float('inf')         # this means "infinity" in Python
    return safe_f

In [79]:
def step(v, direction, step_size):
    """move step_size in the direction from v"""
    return [v_i + step_size * direction_i
            for v_i, direction_i in zip(v, direction)]

In [163]:
    def log_step(iteration,seconds,step_size,value):
        if iteration == 1: print("n\tms\tStep\tValue\n=====================================")
        if iteration % 10 == 0: 
            print("%s\t%s\t%s\t%s" % (iteration,round(seconds*1000,3),step_size,round(value,5)))
        return True

In [168]:
    def minimize_batch(target_fn,  theta, gradient_fn=None,
                       tolerance=0.000001,
                       step_sizes = [100, 10, 1, 0.1, 0.01, 0.001, 0.0001, 0.00001],
                       max_iter = 10000,
                       log_step=log_step):
        """use gradient descent to find theta that minimizes target function"""
        from time import time
        
        target_fn = safe(target_fn)               # safe version of target_fn
        value = target_fn(theta)                  # value we're minimizing
        iteration = 1
        start_time = time()

        if gradient_fn is None:
            gradient_fn = lambda x: estimate_gradient(target_fn,x)
        
        while True:
            
            # new theta candidates for each step size
            gradient = gradient_fn(theta)
            next_thetas = [(step(theta, gradient, -s), s) for s in step_sizes] # add new thetas
            next_thetas = [(target_fn(t[0]),t[0],t[1]) for t in next_thetas] # add new values
            
            # choose the one that minimizes the error function
            next_value, next_theta, current_step = min(next_thetas) # minimizes by first element ie. value
            
            log_ok = log_step(iteration,time()-start_time,current_step,value)
            
            # log iteration, stop if we're "converging"
            if abs(value - next_value) < tolerance:
                print("Reached optimum %.5f after %i iterations" % (value,iteration))
                return theta
            
            if not log_ok or iteration >= max_iter:
                raise RuntimeError("No convergence after %i iterations" % iteration)

            iteration += 1
            theta, value = next_theta, next_value

In [169]:
from numpy.random import randint
v = [randint(-10,10) for i in range(3)]
minimize_batch(sum_of_squares,v,sum_of_squares_gradient)

n	ms	Step	Value
10	0.348	0.1	1.33307
20	0.661	0.1	0.01537
30	0.962	0.1	0.00018
40	1.262	0.1	0.0
Reached optimum 0.00000 after 40 iterations


[0.001163074496311801, 0.0, -0.0008307674973655728]

Gradient Descent kann sehr langsam konvergieren. Hier zwei Beispiele bei denen es gar nicht funktioniert. Für die Rosenbrock-Funktion siehe [Wikipedia](https://en.wikipedia.org/wiki/Gradient_descent)

In [172]:
rosenbrock = lambda x: (1-x[0]**2) + 100*(x[1]-x[0]**2)**2
minimize_batch(rosenbrock,[-.5,.5])


n	ms	Step	Value
10	0.362	0.001	0.60387
20	0.665	0.001	0.57854
30	0.959	0.01	0.55467
40	1.252	0.001	0.52533
50	1.545	0.001	0.49834
60	1.837	0.01	0.46953
70	2.128	0.001	0.43827
80	2.418	0.001	0.40669
90	2.714	0.001	0.37428
100	2.98	0.001	0.3407
110	3.246	0.001	0.30657
120	3.512	0.001	0.20602
130	3.777	0.001	0.16676
140	4.043	0.001	0.12155
150	4.571	0.001	-0.78356
160	4.836	0.001	-0.81584
170	5.091	0.001	-0.83391
180	5.347	0.001	-0.85084
190	5.599	0.01	-0.86765
200	5.852	0.001	-0.88837
210	6.104	0.001	-0.90644
220	6.357	0.0001	-12.78366
230	6.61	0.01	-12.78825
240	6.873	0.001	-12.81026
250	7.152	0.0001	-12.83217
260	7.416	0.01	-12.83675
270	7.667	0.001	-12.85876
280	7.944	1e-05	-23.67241
290	8.199	1e-05	-23.94992
300	8.453	0.0001	-23.9532
310	8.706	0.0001	-23.95416
320	8.959	1e-05	-23.95511
330	9.212	0.0001	-23.95602
340	9.465	0.0001	-23.95697
350	9.717	0.0001	-23.95788
360	9.969	0.0001	-23.95883
370	10.222	0.0001	-23.95978
380	10.487	0.0001	-23.96069
390	10.887	0.0001	-23.96164
400	11.16

RuntimeError: No convergence after 10000 iterations

In [173]:
sinx_x = lambda x: sin(x) / x if x != 0 else 1
minimize_batch(sinx_x,[10])

n	ms	Step	Value
10	0.376	100	inf
20	0.722	100	inf
30	1.057	100	inf
40	1.391	100	inf
50	1.725	100	inf
60	2.057	100	inf
70	2.389	100	inf
80	2.72	100	inf
90	3.045	100	inf
100	3.344	100	inf
110	3.643	100	inf
120	4.167	100	inf
130	4.49	100	inf
140	4.806	100	inf
150	5.111	100	inf
160	5.417	100	inf
170	5.694	100	inf
180	5.97	100	inf
190	6.245	100	inf
200	6.521	100	inf
210	6.796	100	inf
220	7.085	100	inf
230	7.344	100	inf
240	7.617	100	inf
250	7.913	100	inf
260	8.173	100	inf
270	8.433	100	inf
280	8.692	100	inf
290	8.966	100	inf
300	9.234	100	inf
310	9.501	100	inf
320	9.769	100	inf
330	10.042	100	inf
340	10.303	100	inf
350	10.697	100	inf
360	10.977	100	inf
370	11.251	100	inf
380	11.511	100	inf
390	11.783	100	inf
400	12.055	100	inf
410	12.313	100	inf
420	12.573	100	inf
430	12.831	100	inf
440	13.091	100	inf
450	13.35	100	inf
460	13.608	100	inf
470	13.867	100	inf
480	14.125	100	inf
490	14.384	100	inf
500	14.643	100	inf
510	14.901	100	inf
520	15.173	100	inf
530	15.452	100	inf
540	15.711	100	inf
550

RuntimeError: No convergence after 10000 iterations